In [1]:
import os
os.environ["THEANO_FLAGS"] = "floatX=float32,device=cpu"

import deepthought
import os
from deepthought.datasets.eeg.meta_class import DataFileWithMetaClasses

db = DataFileWithMetaClasses(
        filepath='data/OpenMIIR-Perception-512Hz.pklz',
        meta_classes={                                                
                    'within_subject_tuples_group': ['subject', 'condition'],
                    'cross_subject_tuples_group': ['condition'], # cross-participant triplets - this should be very tough                },
                },
    )

In [2]:
from deepthought.datasets.eeg.EEGEpochsDataset import EEGEpochsDataset
INPUT_LEN = 6000
base_dataset = EEGEpochsDataset(
                db=db,
                meta_sources=['subject'],
                selectors={         
                    #'trial_no' : [0,1,  3,4],
                    'condition' : [1],
                },
                use_targets=False,
                stop_sample=INPUT_LEN,
                label_attribute='stimulus_id',
                label_map= { # stimulus_id
                    1: 0, 2: 1, 3: 2, 4: 3, 11: 4, 12: 5, 13: 6, 14: 7, 21: 8, 22: 9, 23: 10, 24: 11,
                },
                remove_dc_offset=True,
                layout='tf',
            )

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36

C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
C:\Users\varun\Anaconda3\lib\site-packages\librosa\cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in v

In [3]:
print (base_dataset.trials.shape)
print (base_dataset.y.shape)
print (base_dataset.metadata[0])
print (base_dataset.metadata[0].keys())

(540, 6000, 1, 64)
(540, 12)
{'subject': 'P01', 'trial_type': 'perception', 'trial_no': 0, 'condition': 1, 'stimulus_id': 1, 'within_subject_tuples_group': 0, 'cross_subject_tuples_group': 0}
dict_keys(['subject', 'trial_type', 'trial_no', 'condition', 'stimulus_id', 'within_subject_tuples_group', 'cross_subject_tuples_group'])


In [4]:
# convert to bc01 axis format
import numpy as np
features_bc01 = np.rollaxis(base_dataset.trials,3,1)
print (features_bc01.shape)

(540, 64, 6000, 1)


In [388]:
# save dataset

import h5py
from fuel.datasets.hdf5 import H5PYDataset

hdf5name = 'data/OpenMIIR-Perception-512Hz.hdf5'
f = h5py.File(hdf5name, mode='w')

N = len(base_dataset.trials)
fx = f.create_dataset('features', features_bc01.shape, dtype='float32')
fy = f.create_dataset('targets', base_dataset.y.shape, dtype='int8')
fx[...] = features_bc01
fy[...] = base_dataset.y

# for using indices
findices = f.create_dataset('indices', (N,), dtype='int')
findices[...] = range(N)

# additional metadata source for hydra-net control flow
subjects = ['P01','P04','P06','P07','P09','P11','P12','P13','P14']
fsubject = f.create_dataset('subjects', (N,), dtype='int8')
fsubject[...] = [subjects.index(meta['subject']) for meta in base_dataset.metadata]

split_dict = {
    'all': {'features': (0, N), 
            'indices': (0, N), 
            'targets': (0, N),
            'subjects': (0, N)
           },
}

f.attrs['split'] = H5PYDataset.create_split_array(split_dict)

f.flush()
f.close()

# metadata for subset selection
import deepthought.util.fs_util as fs_util
fs_util.save(hdf5name+'.meta.pklz', base_dataset.metadata)

C:\Users\varun\Documents\Deep Learning\project\openmiir-rl-2016-master\openmiir-rl-2016-master\fuel\datasets\hdf5.py:270: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ('comment', 'a', comment_len)]))


In [5]:
features_bc01.shape

(540, 64, 6000, 1)

In [6]:
base_dataset.y.shape

(540, 12)

In [7]:
base_dataset.y[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
base_dataset.y[60]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
labels=[]
for i in base_dataset.y:
    labels.append(np.where(i==1))
labels=np.array(labels)
labels=labels.reshape((base_dataset.y.shape[0],))
labels.shape

(540,)

In [10]:
labels[5]

1

In [11]:
labels_dict={0:1,1:2,2:3,3:4,4:11,5:12,6:13,7:14,8:21,9:22,10:23,11:24}

In [12]:
labels

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10,
       10, 10, 10, 10, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,  8,
        8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,
        3,  3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,
        6,  6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
       10, 10, 10, 10, 10, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  1,  1,
        1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,
        8,  8,  8,  8,  9

In [13]:
features=features_bc01

In [14]:
features.shape

(540, 64, 6000, 1)

In [15]:
features=features.reshape(features.shape[0],64,INPUT_LEN)
#features=np.swapaxes(features,1,2)
features.shape

(540, 64, 6000)

In [16]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional,SimpleRNN, Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku

In [100]:
#x_train, x_val, y_train, y_val = train_test_split(features, labels, test_size = 0.3, random_state = 5)
x_val=features[4::5]
y_val=labels[4::5]
x_train=np.delete(features, list(range(0, features.shape[0], 5)), axis=0)
y_train=np.delete(labels, list(range(0, labels.shape[0], 5)), axis=0)

x_test=x_train[3::4]
y_test=y_train[3::4]
x_train=np.delete(x_train, list(range(0, features.shape[0], 4)), axis=0)
y_train=np.delete(y_train, list(range(0, labels.shape[0], 4)), axis=0)


print(x_train.shape,y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  if __name__ == '__main__':


(1296, 64, 3518) (1296,) (432, 64, 3518) (432,) (432, 64, 3518) (432,)


C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  # Remove the CWD from sys.path while we load stuff.


Do not execute the below cell for within-subject evaluation

In [40]:
val=1
val_start=(val-1)*60
val_end=val_start+60

if val==1:
    x_train=features[60:]
    y_train=labels[60:]
elif val==9:
    x_train=features[:480]
    y_train=labels[:480]
else:
    x_train_1=features[:val_start]
    y_train_1=labels[:val_start]
    x_train_2=features[val_end:]
    y_train_2=labels[val_end:]
    x_train=np.concatenate([x_train_1,x_train_2])
    y_train=np.concatenate([y_train_1,y_train_2])
    
x_val=features[val_start:val_end]
y_val=labels[val_start:val_end]
x_test=x_val
y_test=y_val

print(x_train.shape,y_train.shape, x_val.shape, y_val.shape)

(480, 64, 6000) (480,) (60, 64, 6000) (60,)


In [41]:
#x_train=x_train.to_numpy()
x_train = x_train.reshape(x_train.shape[0], 64,INPUT_LEN)
#x_val=x_val.to_numpy()
x_val = x_val.reshape(x_val.shape[0],  64,INPUT_LEN)


In [86]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences = True),input_shape=(64,INPUT_LEN)))
model.add(Dropout(0.7))
model.add(Bidirectional(LSTM(64)))
#model.add(LSTM(100))
model.add(Dropout(0.7))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [88]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_18 (Bidirectio (None, 64, 128)           3105280   
_________________________________________________________________
dropout_27 (Dropout)         (None, 64, 128)           0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_28 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_29 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 12)               

In [128]:
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=1, batch_size=32)

Train on 480 samples, validate on 60 samples
480/480 [==============================] - 25s 53ms/sample - loss: 0.0281 - accuracy: 1.0000 - val_loss: 3.7402 - val_accuracy: 0.3000


In [129]:
yhat=model.predict(x_test)
yhat=yhat.round()
yhat=np.argmax(yhat,axis = 1)
print('Accuracy:',np.sum(yhat==y_test)/y_test.size*100)

Accuracy: 31.666666666666664


In [130]:
cm = np.zeros((12,12), dtype=int)
np.add.at(cm, [y_test, yhat], 1)
cm

C:\Users\varun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


array([[1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0],
       [1, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3],
       [2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0],
       [0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 1],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 2, 0, 0],
       [1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2],
       [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3]])